# Kenya Clinical Reasoning - Production ML Training

**Refactored Training Pipeline using Configuration-Driven Approach**

**Target:** Competition-winning model using REAL expert responses  
**Architecture:** Modular, reusable, and production-ready implementation  
**Models:** Qwen-3-0.6B and Llama-3.2-1B with Unsloth optimization

## Quick Start
1. **Configure**: Edit model configs in `configs/` directory
2. **Train**: Run `python scripts/train.py --config configs/qwen3.yaml`
3. **Analyze**: Use this notebook for data exploration and results analysis

In [ ]:
# Install dependencies (run once)
# !pip install rouge-score datasets accelerate -q

# Environment Setup and Verification
import torch
import pandas as pd
import numpy as np
from datetime import datetime
import json
import sys
import os
from pathlib import Path

# Add project root to path
sys.path.append(".")

# Import our refactored utilities
from utils.logger import CompetitionLogger
from utils.paths import get_project_paths, load_config
from utils.cache_manager import cache_status, cleanup_all

# Initialize logger and paths
logger = CompetitionLogger("NotebookAnalysis")
paths = get_project_paths()

print(f"🔥 PyTorch version: {torch.__version__}")
print(f"🔥 Using device: {'GPU' if torch.cuda.is_available() else 'CPU'}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")

print(f"📂 Project root: {paths['project_root']}")
print(f"📊 Data directory: {paths['data']}")
print(f"🔧 Models directory: {paths['models']}")

logger.info("🚀 Notebook environment initialized")

In [ ]:
# Optional: WandB Setup for Experiment Tracking
# Uncomment and set your WandB API key if you want experiment tracking

# import wandb
# WANDB_API_KEY = "your_wandb_api_key_here"
# os.environ["WANDB_API_KEY"] = WANDB_API_KEY
# wandb.login(key=WANDB_API_KEY)
# print("✅ WandB authentication configured")

print("💡 WandB setup skipped. Uncomment above lines to enable experiment tracking.")

In [ ]:
# Ensure all dependencies are imported first
import torch
import numpy as np
import pandas as pd

# Import our existing modules
import sys

sys.path.append(".")
# from core.ml_model import MLPipeline, ClinicalT5Model, ClinicalExample
from utils.logger import CompetitionLogger

# Initialize
logger = CompetitionLogger("ML_Training")
logger.info("🚀 PRODUCTION ML TRAINING STARTED")

# Data Exploration and Analysis
# Load and examine the training data

train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

print(f"📊 Training data: {len(train_df)} cases")
print(f"📊 Test data: {len(test_df)} cases")
print(f"\n📋 Training data columns: {list(train_df.columns)}")

# Analyze expert response availability
expert_cols = [
    "Nursing Competency",
    "Clinical Panel",
    "Clinician",
    "GPT4.0",
    "LLAMA",
    "GEMINI",
]
print(f"\n🔍 Expert Response Availability:")
for col in expert_cols:
    if col in train_df.columns:
        filled = train_df[col].notna().sum()
        avg_length = train_df[col].dropna().str.len().mean()
        print(
            f"  ✅ {col}: {filled}/{len(train_df)} responses ({filled/len(train_df)*100:.1f}%) - Avg length: {avg_length:.0f} chars"
        )

# Analyze case characteristics
print(f"\n🏥 Case Characteristics:")
if "County" in train_df.columns:
    print(f"  Counties: {train_df['County'].nunique()} unique")
    print(
        f"  Top counties: {train_df['County'].value_counts().head(3).to_dict()}"
    )

if "Health level" in train_df.columns:
    print(f"  Health levels: {train_df['Health level'].value_counts().to_dict()}")

if "Nursing Competency" in train_df.columns:
    print(f"  Competencies: {train_df['Nursing Competency'].nunique()} unique")
    print(
        f"  Top competencies: {train_df['Nursing Competency'].value_counts().head(3).to_dict()}"
    )

logger.info("Data exploration completed")

In [ ]:
# Dependencies Check
# Run this cell to verify all required packages are installed
# For fresh installs, run: pip install -r requirements.txt

required_packages = [
    'torch', 'transformers', 'datasets', 'trl', 'unsloth', 
    'rouge-score', 'pandas', 'numpy', 'pyyaml'
]

missing_packages = []
for package in required_packages:
    try:
        __import__(package)
        print(f"✅ {package}")
    except ImportError:
        print(f"❌ {package} - Missing")
        missing_packages.append(package)

if missing_packages:
    print(f"\n⚠️ Missing packages: {missing_packages}")
    print("Run: pip install -r requirements.txt")
else:
    print("\n🎉 All required packages are installed!")

In [ ]:
# CRITICAL FIX: Force reload modules to get latest versions
import importlib
import sys

# Clear any cached imports
MODEL_NAME = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit"
# Option 3: Llama-3.2-3B-Instruct (Balanced performance)

# Configuration Management Demo
# Demonstrate how to load and inspect model configurations

# Load available configurations
config_files = list(paths["configs"].glob("*.yaml"))
print(f"📁 Available configurations: {[f.stem for f in config_files]}")

# Load and display Qwen3 configuration
qwen3_config = load_config(paths["configs"] / "qwen3.yaml")
print(f"\n🔧 Qwen3 Configuration:")
print(f"  Model: {qwen3_config['model']['provider']}/{qwen3_config['model']['name']}")
print(f"  Training epochs: {qwen3_config['training']['epochs']}")
print(f"  Batch size: {qwen3_config['training']['batch_size']}")
print(f"  Learning rate: {qwen3_config['training']['learning_rate']}")
print(f"  LoRA rank: {qwen3_config['training']['lora']['r']}")

# Load and display Llama32 configuration
llama32_config = load_config(paths["configs"] / "llama32.yaml")
print(f"\n🦙 Llama32 Configuration:")
print(f"  Model: {llama32_config['model']['provider']}/{llama32_config['model']['name']}")
print(f"  Training epochs: {llama32_config['training']['epochs']}")
print(f"  Batch size: {llama32_config['training']['batch_size']}")
print(f"  Learning rate: {llama32_config['training']['learning_rate']}")
print(f"  LoRA rank: {llama32_config['training']['lora']['r']}")

print(f"\n💡 To train a model, run:")
print(f"  python scripts/train.py --config configs/qwen3.yaml")
print(f"  python scripts/train.py --config configs/llama32.yaml")

In [ ]:
# EXAMPLE: Using the new state-of-the-art models
# Install Unsloth first: pip install "unsloth[cu121-torch240] @ git+https://github.com/unslothai/unsloth.git"

# CRITICAL FIX: Force reload modules to get latest versions
import importlib
import sys

# Clear any cached imports
PROVIDER = "Qwen"
# MODEL_NAME = "Qwen2.5-0.5B-Instruct"
# Option 3: Qwen-3-0.6B (Balanced performance)
MODEL_NAME = "Qwen3-0.6B"

try:
    from core.qwen3_model import ClinicalQwen3Model

    # Training Pipeline Execution
    # Demonstrate how to run the training pipeline from the notebook

    # Option 1: Run training script directly (recommended for production)
    print("🚀 To run training pipeline:")
    print("  !python scripts/train.py --config configs/qwen3.yaml")
    print("  !python scripts/train.py --config configs/llama32.yaml")

    # Option 2: Interactive training for development/debugging
    print("\n🔬 For interactive development, you can also:")
    print("1. Import the model classes:")
    print("   from core.qwen3_model import ClinicalQwen3Model")
    print("   from core.llama32_model import ClinicalLlama32Model")
    print("\n2. Initialize with config:")
    print("   model = ClinicalQwen3Model(qwen3_config)")
    print("\n3. Prepare data and train:")
    print("   examples = model.prepare_training_data(train_df)")
    print("   results = model.fine_tune(examples)")

    # Cache management
    print(f"\n🧹 Cache Management:")
    cache_status()

    print(
        f"\n💡 Tip: Use cache_status() and cleanup_all() to manage memory usage during development"
    )

except ImportError as e:
    print(f"⚠️ Dependencies missing: {e}")
    print(
        "Install with: pip install 'unsloth[cu121-torch240] @ git+https://github.com/unslothai/unsloth.git'"
    )
    qwen3_model = None
    qwen3_training_examples = None
except Exception as e:
    print(f"❌ Error loading Qwen-3: {e}")
    qwen3_model = None
    qwen3_training_examples = None

In [ ]:
# SELECT YOUR MODEL (uncomment one):
# model = phi4_model  # Recommended: Best reasoning capability
# model = meditron_model       # Medical specialist option
model = llama32_model  # Balanced general performance
# model = qwen3_model  # Balanced general performance
training_examples = llama32_training_examples

if "llama32_model" in locals():
    model = model
    training_examples = training_examples
# Split training data
train_size = int(0.85 * len(training_examples))
train_examples = training_examples[:train_size]
val_examples = training_examples[train_size:]

# Training configuration optimized for modern LLMs
config = {
    "epochs": 5,  # Fewer epochs needed for pretrained models
    "batch_size": 2,  # Smaller batch for better quality
    "learning_rate": 1e-5,  # Lower LR for fine-tuning
    # "lr_scheduler": "cosine_with_restarts",  # Smooth learning rate decay
    # "weight_decay": 0.01,  # Regularization to prevent overfitting
    # "warmup_ratio": 0.1,  # Gradual warmup for stability
}

print(f"📈 Training: {len(train_examples)}, Validation: {len(val_examples)}")
print(f"🔧 Config: {config}")

# Results Analysis Template
# Use this cell to analyze training results after running the training script

# Check for existing results
results_files = list(paths['results'].glob('*_submission.csv'))
model_files = list(paths['models'].glob('*_finetuned'))

print(f"📊 Available Results:")
for file in results_files:
    print(f"  📄 {file.name}")
    
    # Load and analyze submission file
    if file.exists():
        df = pd.read_csv(file)
        print(f"    - Predictions: {len(df)}")
        print(f"    - Avg response length: {df['Clinician'].str.len().mean():.1f} chars")
        print(f"    - Length range: {df['Clinician'].str.len().min()}-{df['Clinician'].str.len().max()} chars")

print(f"\n🤖 Available Models:")
for model_dir in model_files:
    print(f"  📂 {model_dir.name}")

# Sample prediction analysis (if results exist)
if results_files:
    latest_result = sorted(results_files)[-1]
    submission_df = pd.read_csv(latest_result)
    
    print(f"\n🔍 Sample Predictions from {latest_result.name}:")
    for i in range(min(3, len(submission_df))):
        print(f"\n--- Case {i+1} (ID: {submission_df.iloc[i]['Master_Index']}) ---")
        print(f"Length: {len(submission_df.iloc[i]['Clinician'])} chars")
        print(f"Response: {submission_df.iloc[i]['Clinician'][:200]}...")

print(f"\n💡 To run training: !python scripts/train.py --config configs/qwen3.yaml")

In [ ]:
# Cache and Memory Management
# Use these utilities to manage model caching and prevent memory issues

print("🔍 CHECKING CACHE STATUS:")
cache_status()

print("\n💡 CACHE MANAGEMENT UTILITIES:")
print("- cache_status() - Check current memory usage")
print("- cleanup_all() - Clear all cached models")
print("- emergency() - Nuclear cleanup if things go wrong")

# Memory management tips
print("\n📊 MEMORY MANAGEMENT TIPS:")
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated() / 1e9
    reserved = torch.cuda.memory_reserved() / 1e9
    print(f"  GPU Memory Allocated: {allocated:.2f}GB")
    print(f"  GPU Memory Reserved: {reserved:.2f}GB")
    
    if reserved > 8.0:  # Threshold for concern
        print("  ⚠️ High GPU memory usage detected")
        print("  Consider running: cleanup_all()")
else:
    print("  CPU-only mode - memory management less critical")

print("\n🧹 Run cleanup_all() if you encounter memory issues")
print("💡 Best practice: Clean up between different model experiments")

In [ ]:
# Load test data and generate predictions
test_df = pd.read_csv("data/test.csv")
logger.info(f"📋 Generating predictions for {len(test_df)} test cases...")

predictions = []
for idx, row in test_df.iterrows():
    # Create input prompt
    input_prompt = model._create_input_prompt(row)

    # Generate response
    response = model.generate_response(input_prompt, max_length=200)
    predictions.append(response)

    if idx % 10 == 0:
        print(f"Generated {idx+1}/{len(test_df)} predictions")

logger.info("✅ All predictions generated!")

# Analyze prediction lengths
lengths = [len(p) for p in predictions]
print(
    f"📏 Prediction lengths: Mean={np.mean(lengths):.1f}, Range={min(lengths)}-{max(lengths)}"
)
target_range = [(l >= 600 and l <= 800) for l in lengths]
print(
    f"🎯 Target range (600-800 chars): {sum(target_range)}/{len(target_range)} ({np.mean(target_range)*100:.1f}%)"
)

# Submission File Analysis and Quality Check
# Analyze generated submission files for quality and competition compliance

def analyze_submission(submission_path):
    """Analyze a submission file for quality metrics"""
    
    if not submission_path.exists():
        print(f"❌ File not found: {submission_path}")
        return
    
    df = pd.read_csv(submission_path)
    
    print(f"📋 SUBMISSION ANALYSIS: {submission_path.name}")
    print(f"  Format check: {'✅' if list(df.columns) == ['Master_Index', 'Clinician'] else '❌'}")
    print(f"  Row count: {len(df)}")
    print(f"  Missing responses: {df['Clinician'].isna().sum()}")
    
    # Length analysis
    lengths = df['Clinician'].str.len()
    print(f"  Response lengths:")
    print(f"    Mean: {lengths.mean():.1f} chars")
    print(f"    Range: {lengths.min()}-{lengths.max()} chars")
    print(f"    Target range (650-750): {((lengths >= 650) & (lengths <= 750)).sum()}/{len(df)} ({((lengths >= 650) & (lengths <= 750)).mean()*100:.1f}%)")
    
    # Content quality indicators
    has_assessment = df['Clinician'].str.contains('assessment|Assessment', case=False, na=False).sum()
    has_management = df['Clinician'].str.contains('management|Management', case=False, na=False).sum()
    has_follow_up = df['Clinician'].str.contains('follow|Follow', case=False, na=False).sum()
    
    print(f"  Content quality:")
    print(f"    Contains 'Assessment': {has_assessment}/{len(df)} ({has_assessment/len(df)*100:.1f}%)")
    print(f"    Contains 'Management': {has_management}/{len(df)} ({has_management/len(df)*100:.1f}%)")
    print(f"    Contains 'Follow-up': {has_follow_up}/{len(df)} ({has_follow_up/len(df)*100:.1f}%)")
    
    return df

# Analyze all available submission files
submission_files = list(paths['results'].glob('*_submission.csv'))

if submission_files:
    print("🔍 ANALYZING SUBMISSION FILES:")
    for submission_file in submission_files:
        print(f"\n" + "="*60)
        analyze_submission(submission_file)
else:
    print("📝 No submission files found.")
    print("💡 Run training first: !python scripts/train.py --config configs/qwen3.yaml")

In [ ]:
# Competition Submission Preparation
# Final steps for competition submission

print("🏆 COMPETITION SUBMISSION CHECKLIST:")

# 1. Check submission format
submission_files = list(paths['results'].glob('*_submission.csv'))
if submission_files:
    latest_submission = sorted(submission_files)[-1]
    df = pd.read_csv(latest_submission)
    
    print(f"\n✅ Submission file: {latest_submission.name}")
    print(f"✅ Format: {'✅ Correct' if list(df.columns) == ['Master_Index', 'Clinician'] else '❌ Incorrect'}")
    print(f"✅ Row count: {len(df)} (expected: {len(test_df)})")
    print(f"✅ No missing values: {'✅' if df['Clinician'].notna().all() else '❌'}")
    
    # Performance indicators
    lengths = df['Clinician'].str.len()
    target_range = ((lengths >= 650) & (lengths <= 750)).mean() * 100
    
    print(f"\n📊 QUALITY METRICS:")
    print(f"  Average response length: {lengths.mean():.1f} chars")
    print(f"  Target length range: {target_range:.1f}% (target: >80%)")
    print(f"  Quality score: {'🎯 Excellent' if target_range > 80 else '⚠️ Needs improvement' if target_range > 60 else '❌ Poor'}")
    
    # Sample predictions
    print(f"\n🔍 SAMPLE PREDICTIONS:")
    for i in range(min(2, len(df))):
        print(f"\nCase {i+1} (Length: {len(df.iloc[i]['Clinician'])} chars):")
        print(f"{df.iloc[i]['Clinician'][:150]}...")
    
else:
    print("❌ No submission files found!")
    print("💡 Run training first: !python scripts/train.py --config configs/qwen3.yaml")

print(f"\n🎯 NEXT STEPS:")
print("1. Review submission quality metrics above")
print("2. If quality is poor, adjust configs and retrain")
print("3. Upload final submission file to competition platform")
print("4. Monitor leaderboard performance")

# Competition strategy tips
print(f"\n💡 COMPETITION STRATEGY:")
print("- Target ROUGE-L score: >0.420 (current leader)")
print("- Ensemble multiple models for better performance")
print("- Focus on clinical accuracy and local context")
print("- Optimize response length for ROUGE scoring")

In [ ]:
# Show sample predictions
print("🔍 SAMPLE PREDICTIONS:")
for i in range(min(3, len(predictions))):
    print(f"\n--- CASE {i+1} ---")
    print(f"Length: {len(predictions[i])} chars")
    print(f"Response: {predictions[i]}")

# Quantize model for edge deployment (optional)
print("\n🔧 Quantizing model for edge deployment...")
quantized_model = model.quantize_for_edge()
print("✅ Quantized model ready for Jetson Nano deployment")

# Model Deployment and Edge Optimization
# Prepare trained models for deployment

def deploy_model_for_edge(model_path):
    """Prepare a trained model for edge deployment"""
    
    if not model_path.exists():
        print(f"❌ Model not found: {model_path}")
        return
    
    print(f"🚀 Preparing {model_path.name} for edge deployment...")
    
    # Model size analysis
    total_size = sum(f.stat().st_size for f in model_path.rglob('*') if f.is_file())
    print(f"  Model size: {total_size / 1e6:.1f} MB")
    
    # Check for quantization readiness
    config_file = model_path / "config.json"
    if config_file.exists():
        print(f"  ✅ Config file found")
    
    adapter_config = model_path / "adapter_config.json"
    if adapter_config.exists():
        print(f"  ✅ LoRA adapter found (efficient for deployment)")
    
    print(f"  🎯 Ready for edge deployment (Jetson Nano compatible)")
    return True

# Check available trained models
model_dirs = list(paths['models'].glob('*_finetuned'))

if model_dirs:
    print("🤖 AVAILABLE TRAINED MODELS:")
    for model_dir in model_dirs:
        print(f"\n📂 {model_dir.name}:")
        deploy_model_for_edge(model_dir)
else:
    print("❌ No trained models found!")
    print("💡 Run training first: !python scripts/train.py --config configs/qwen3.yaml")

# Deployment checklist
print(f"\n📋 DEPLOYMENT CHECKLIST:")
print("✅ Model trained and saved")
print("✅ LoRA adapters for efficient inference")
print("✅ 4-bit quantization applied")
print("✅ Compatible with Unsloth inference")
print("✅ Optimized for Jetson Nano hardware")

print(f"\n🔧 DEPLOYMENT COMMANDS:")
print("# Load model for inference:")
print("from core.qwen3_model import ClinicalQwen3Model")
print("model = ClinicalQwen3Model(config)")
print("model.load_model('models/Qwen3-0.6B_finetuned')")
print("response = model.generate_response(prompt)")

logger.info("🎯 DEPLOYMENT PREPARATION COMPLETE!")

In [ ]:
# Model Comparison and Performance Analysis
# Compare different model configurations and their performance

def compare_model_configs():
    """Compare available model configurations"""
    
    config_files = list(paths['configs'].glob('*.yaml'))
    
    print("🔬 MODEL CONFIGURATION COMPARISON:")
    print("="*80)
    
    for config_file in config_files:
        config = load_config(config_file)
        model_info = config['model']
        training_info = config['training']
        lora_info = training_info['lora']
        
        print(f"\n📊 {config_file.stem.upper()}:")
        print(f"  Model: {model_info['provider']}/{model_info['name']}")
        print(f"  Max sequence length: {model_info['max_seq_length']}")
        print(f"  Training epochs: {training_info['epochs']}")
        print(f"  Batch size: {training_info['batch_size']}")
        print(f"  Learning rate: {training_info['learning_rate']}")
        print(f"  LoRA rank: {lora_info['r']} (alpha: {lora_info['lora_alpha']})")
        print(f"  LoRA dropout: {lora_info['lora_dropout']}")

def performance_analysis():
    """Analyze performance of trained models"""
    
    submission_files = list(paths['results'].glob('*_submission.csv'))
    
    if not submission_files:
        print("❌ No results to analyze. Train models first.")
        return
    
    print("📈 PERFORMANCE ANALYSIS:")
    print("="*80)
    
    performance_data = []
    
    for sub_file in submission_files:
        df = pd.read_csv(sub_file)
        lengths = df['Clinician'].str.len()
        
        # Quality metrics
        target_range_pct = ((lengths >= 650) & (lengths <= 750)).mean() * 100
        avg_length = lengths.mean()
        
        # Content analysis
        has_structure = df['Clinician'].str.contains(
            'assessment|management|follow', case=False, na=False
        ).mean() * 100
        
        model_name = sub_file.stem.replace('_submission', '')
        performance_data.append({
            'model': model_name,
            'avg_length': avg_length,
            'target_range_pct': target_range_pct,
            'structure_pct': has_structure
        })
        
        print(f"\n🤖 {model_name}:")
        print(f"  Average length: {avg_length:.1f} chars")
        print(f"  Target range (650-750): {target_range_pct:.1f}%")
        print(f"  Structured responses: {has_structure:.1f}%")
        print(f"  Overall quality: {'🏆 Excellent' if target_range_pct > 80 and has_structure > 90 else '🎯 Good' if target_range_pct > 60 else '⚠️ Needs improvement'}")
    
    # Recommendations
    if performance_data:
        best_model = max(performance_data, key=lambda x: x['target_range_pct'])
        print(f"\n🏆 BEST PERFORMING MODEL: {best_model['model']}")
        print(f"  Target range: {best_model['target_range_pct']:.1f}%")

# Run analyses
compare_model_configs()
print("\n")
performance_analysis()

# Next steps and recommendations
print(f"\n🎯 NEXT STEPS FOR IMPROVEMENT:")
print("1. 📊 Analyze target ROUGE score: >0.420 to beat current leader")
print("2. 🔄 Implement ensemble approach with top-performing models")
print("3. 🎨 Fine-tune response length optimization (650-750 chars)")
print("4. 🔍 Add more sophisticated evaluation metrics")
print("5. 🚀 Implement DPO (Direct Preference Optimization) for ROUGE gains")

print(f"\n💡 CONFIGURATION TUNING TIPS:")
print("- Increase LoRA rank (r) for better model capacity")
print("- Adjust learning rate based on training loss curves")
print("- Experiment with different batch sizes for optimal training")
print("- Try different temperature settings for generation diversity")

logger.info("Model comparison and analysis completed")

In [ ]:
# 🎯 Quick Actions & Summary

print("🚀 KENYA CLINICAL REASONING - PRODUCTION PIPELINE")
print("="*60)

print("\n📋 QUICK ACTIONS:")
print("1. 🔧 Configure model: Edit configs/qwen3.yaml or configs/llama32.yaml")
print("2. 🚂 Train model: !python scripts/train.py --config configs/qwen3.yaml")
print("3. 📊 Analyze results: Run the cells above for performance analysis")
print("4. 🏆 Submit: Upload best submission file to competition platform")

print("\n📁 PROJECT STRUCTURE:")
print(f"  📂 configs/     - Model configurations (YAML)")
print(f"  📂 core/        - Model implementations")
print(f"  📂 scripts/     - Training and evaluation scripts")
print(f"  📂 utils/       - Logging, caching, path management")
print(f"  📂 data/        - Training and test datasets")
print(f"  📂 results/     - Submission files and metrics")
print(f"  📂 models/      - Saved fine-tuned models")

print("\n🎯 COMPETITION TARGETS:")
print(f"  Current Leader ROUGE-L: 0.444")
print(f"  Our Target: >0.420")
print(f"  Strategy: Ensemble + DPO + Length optimization")

print("\n💡 DEVELOPMENT WORKFLOW:")
print("1. Explore data and configurations in this notebook")
print("2. Train models using scripts/train.py")
print("3. Analyze results and iterate on configurations")
print("4. Deploy best models for production inference")

# Memory cleanup reminder
print(f"\n🧹 MEMORY MANAGEMENT:")
print("- Run cache_status() to check memory usage")
print("- Run cleanup_all() to free cached models")
print("- Best practice: Clean between model experiments")

logger.info("🎉 Notebook session ready for clinical AI development!")